# OpenCV Detection and Tracking
Based on: https://pysource.com/2021/10/05/object-tracking-from-scratch-opencv-and-python/

Using YOLOv4 for object detection and tracking.

In [2]:
import cv2

In [2]:
import cv2
import numpy as np
from object_detection import ObjectDetection
import math
from tracker import *

import cv2
import re

cv_info = [re.sub('\s+', ' ', ci.strip()) for ci in cv2.getBuildInformation().strip().split('\n') 
               if len(ci) > 0 and re.search(r'(nvidia*:?)|(cuda*:)|(cudnn*:)', ci.lower()) is not None]

if len(cv_info) > 0:
    print('\n'.join(cv_info))
    print('\n')
else:
    print('No CUDA/OpenCV GPU found')
    print('\n')
    raise Exception('No CUDA/OpenCV GPU found')

# Initialize Object Detection
od = ObjectDetection()

cap = cv2.VideoCapture("los_angeles.mp4")
#cap = cv2.VideoCapture(0) # for using WebCam

# Create tracker object
tracker = EuclideanDistTracker()

# Read first frame
ok, frame = cap.read()
if not ok:
    print ('Cannot read video file')
    sys.exit()

# Extract Region of interest ROI
roi_coordinates = cv2.selectROI(frame, False)
cv2.destroyAllWindows()
print("Region of interest: ", roi_coordinates)

while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape    

    # 1. Object Detection
    roi = frame[int(roi_coordinates[1]):int(roi_coordinates[1] + roi_coordinates[3]), int(roi_coordinates[0]):int(roi_coordinates[0] + roi_coordinates[2])]
    
    (class_ids, scores, boxes) = od.detect(roi)

    # 2. Object Tracking
    boxes_ids = tracker.update(boxes)
    for box_id in boxes_ids:
        x, y, w, h, id, direction = box_id
        cv2.putText(roi, str(id) + " " + format(direction,".2f"), (x, y - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

    cv2.imshow("roi", roi)
    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

Extra dependencies: cudart_static.lib nppc.lib nppial.lib nppicc.lib nppidei.lib nppif.lib nppig.lib nppim.lib nppist.lib nppisu.lib nppitc.lib npps.lib cublas.lib cudnn.lib cufft.lib -LIBPATH:C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.0/lib/x64
NVIDIA CUDA: YES (ver 11.0, CUFFT CUBLAS FAST_MATH)
NVIDIA GPU arch: 61
NVIDIA PTX archs:
cuDNN: YES (ver 8.0.5)


Loading Object Detection
Running opencv dnn with YOLOv4
Region of interest:  (477, 562, 652, 292)
